In [1]:
org_name = "my_code.py"

# Read the contents of the file into a string
with open(org_name, "r") as file:
    python_code = file.read()

# Print the content as a string
print(python_code)

from typing import List, Tuple


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    for idx, elem in enumerate(numbers):
        for idx2, elem2 in enumerate(numbers):
            if idx != idx2:
                distance = abs(elem - elem2)
                if distance < threshold:
                    return True

    return False

def separate_paren_groups(paren_string: str) -> List[str]:
    result = []
    current_string = []
    current_depth = 0

    for c in paren_string:
        if c == '(':
            current_depth += 1
            current_string.append(c)
        elif c == ')':
            current_depth -= 1
            current_string.append(c)

            if current_depth == 0:
                result.append(''.join(current_string))
                current_string.clear()

    return result

def truncate_number(number: float) -> float:
    return number % 1.0

def below_zero(operations: List[int]) -> bool:
    balance = 0

    for op in operations:
   

In [2]:
code_split = python_code.split("\ndef")
python_fcts = [code_split[0] + "\ndef" + code_piece for code_piece in code_split[1:]]

In [3]:
def get_function_name(func_string):
    # Extract the function definition substring
    start_index = func_string.find("def ")
    end_index = func_string.find("(")
    if start_index == -1 or end_index == -1:
        raise ValueError("No function definition found in the provided string.")

    # Extract the function definition substring
    function_definition = func_string[start_index:end_index]
    function_name = function_definition.split("(")[0][4:]
    return function_name

def execute_function_from_string(func_string, *args, **kwargs):
    # Execute the function definition in the current scope
    exec(func_string)

    # Get the function from locals    
    function_name = get_function_name(func_string)
    function_to_call = locals().get(function_name)

    # Check if the function exists
    if function_to_call is not None:
        # Call the function with arguments and return the result
        return function_to_call(*args, **kwargs)
    else:
        raise ValueError("No function named 'function' found in the provided string.")

In [4]:
from openai_code import explain_code, generate_inputs, generate_unittest_advanced

import yaml

with open('config.yml', 'r') as file:
    config = yaml.safe_load(file)
openai_key = config["openai_key"]

In [5]:
for fct in python_fcts:
    print(fct)
    code_explanation = explain_code(fct, openai_key)
    input_list = generate_inputs(fct, code_explanation, openai_key)
    print(input_list)
    output_list = [0]*len(eval(input_list))
    for i, input in enumerate(eval(input_list)):
        try:
            output_list[i] = execute_function_from_string(fct, *input)
        except TypeError:
            output_list[i] = execute_function_from_string(fct, input)
    unittest = generate_unittest_advanced(fct, code_explanation, input_list, output_list, openai_key)
    print(unittest)
    
    function_name = get_function_name(fct)
    file_name = f"tests/test_{function_name}.py"
    complete_unittest = code_split[0] + f"\nfrom ..{org_name.split(".")[0]} import {function_name}\n" + unittest
    with open(file_name, "w") as file:
        file.write(complete_unittest)

    print(f"Python code has been written to '{file_name}'")

from typing import List, Tuple


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    for idx, elem in enumerate(numbers):
        for idx2, elem2 in enumerate(numbers):
            if idx != idx2:
                distance = abs(elem - elem2)
                if distance < threshold:
                    return True

    return False

[( [1.0, 2.0, 3.0], 0.5), ([10.5, 15.3, 20.7], 5.0), ([-3.5, -2.0, 0.0, 1.5], 1.0), ([5.5, 6.0, 6.5, 7.0], 0.2), ([100.0, 105.0, 110.0, 115.0], 10.0)]
import unittest

class TestHasCloseElements(unittest.TestCase):

    def test_has_close_elements(self):
        self.assertEqual(has_close_elements([1.0, 2.0, 3.0], 0.5), False)
        self.assertEqual(has_close_elements([10.5, 15.3, 20.7], 5.0), True)
        self.assertEqual(has_close_elements([-3.5, -2.0, 0.0, 1.5], 1.0), False)
        self.assertEqual(has_close_elements([5.5, 6.0, 6.5, 7.0], 0.2), False)
        self.assertEqual(has_close_elements([100.0, 105.0, 110.0, 115.0], 10.